# A Fast Introduction to GAN by Ege Beyazit
In this notebook, we study GAN: Generative Adversarial Networks (Goodfelllow, 2014). We try to highlight the most important properties in order to jump into GAN research ASAP.

The list of references we use for this notebook is:
1. NIPS'16 Tutorial on how GANs work: https://arxiv.org/abs/1701.00160
2. An Annotated Proof of GANs: https://srome.github.io/An-Annotated-Proof-of-Generative-Adversarial-Networks-with-Implementation-Notes/
3. NIPS'16 Tutorial on how to train GANs: https://github.com/soumith/ganhacks
4. Learning Class-Conditional GANs with Active Sampling, KDD'19: https://www.kdd.org/kdd2019/accepted-papers/view/learning-class-conditional-gans-with-active-sampling
5. Improved Techniques for Training GANs (1000+ citations): https://arxiv.org/pdf/1606.03498.pdf
## 0. Why use GANs?
GANs were designed to avoid some common disadvantages of other generative models. Specifically:
1. They can generate samples in parallel, unlike the models like FVBNs that require sequential sampling to converge.
2. The generator function has very few restrictions unlike RBMs (can only use the distributions that admits Markov Chain sampling) and non-linear ICAs (generator must be invertible and latent code must have the same dimensionality as the input).
3. No Markov Chains are needed, unlike RBMs and GSNs.
4. No variational bound is needed. Some model families can be used with GANs are known to be asymptotically consistent -> universal approximators.
5. GANs usually produce better samples than other generators.

## 1. How do GANs work?
1. GAN framework is a game between two players: 
  - *generator*: tries to create samples that look like the training data, and 
  - *discriminator*: tries to determine whether the samples are real or fake.

2. GANs are structured (z -> x, directed where every latent variable influences every observed variable) probabilistic models containing latent variables $\mathbf{z}$ and observed variables $\mathbf{x}$.
3. The two players are represented by two functions. Both of them are differentiable w.r.t. both the inputs and their parameters:
  - Generator: $G(\mathbf{z}; \mathbf{\theta}^{(G)})$, where $\mathbf{z}$ is input noise and $\mathbf{\theta}^{(G)}$ are the model parameters,
  - Discriminator: $D(\mathbf{x}; \mathbf{\theta}^{(D)})$, where $\mathbf{x}$ are the observed variables and $\mathbf{\theta}^{(D)}$ are the model parameters. 
4. Both players try to minimize their own cost functions:
   - Generator: $J^{(G)}(\mathbf{\theta}^{(D)}, \mathbf{\theta}^{(G)})$ only having control on $G$ parameters for minimization, 
   - Discriminator: $J^{(D)}(\mathbf{\theta}^{(D)}, \mathbf{\theta}^{(G)})$ only having control on $D$ parameters for minimization.
   - **Therefore this is a game, more than an optimization problem.**
4. The solution to an optimization problem is a local minima, the solution to a game is *Nash equilibria*. **Specifically for GANs, the Nash equilibrium is a tuple $(\mathbf{\theta}^{(D)}, \mathbf{\theta}^{(G)})$ that is a local minimum of $J^{(D)}$ with respect to $\mathbf{\theta}^{(D)}$ and a local minimum of $J^{(G)}$ with respect to $\mathbf{\theta}^{(G)}$.**

### 1.1. The Generator
1. When $\mathbf{z}$ is sampled from some prior distribution, $G(\mathbf{z})$ yields a sample of $\mathbf{x}$ drawn from $p_{model}$. **$G$ is typically a deep neural network.**
2. Inputs to the function $G$ can be to any layer of the network that represents it. For example,
  - We can partition $\mathbf{z}$ into two vectors $\mathbf{z}^{(1)}$ and  $\mathbf{z}^{(2)}$, then feed $\mathbf{z}^{(1)}$ to the input and $\mathbf{z}^{(2)}$ to the output layer. If $\mathbf{z}^{(2)}$ is Gaussian, this makes $\mathbf{x}$ conditionally Gaussian given $\mathbf{z}^{(1)}$.
3. If we want $p_{model}$ to have full support on $\mathbf{x}$ space, we need the dimension of $\mathbf{z}$ to be at least as large as the dimension of $\mathbf{x}$, and $G$ must be differentiable.

### 1.2. The Training Process
Simultaneous SGD, for each step:
  1. Sample a minibatch of $\mathbf{x}$ values from the dataset and,
  2. Sample a minibatch of $\mathbf{z}$ values drawn from the model's prior over latent variables.
  3. Do both gradient steps simultaneously: update $\mathbf{\theta}^{(D)}$ to reduce $J^{(D)}$, and update $\mathbf{\theta}^{(G)}$ to reduce $J^{(G)}$.

## 2. Cost Functions
### 2.1. Discriminator Cost $J^{(D)}$
1.  All games designed for GANs use the same cost for discriminator.
2.  The cost for the discriminator is:
\begin{equation*}
J^{(D)}(\mathbf{\theta}^{(D)}, \mathbf{\theta}^{(G)}) = 
-\frac{1}{2}\mathbb{E}_{\mathbf{x} \sim p_{data}}\log D(\mathbf{x}) 
-\frac{1}{2}\mathbb{E}_{\mathbf{z}}\log(1-D(G(\mathbf{z}))).
\end{equation*}
Note that this is a straightforward negative log likelihood function that receives data from two different sources. The objective is maximizing the likelihood when data from original distribution is received, and minimizing it when data from the generator is received. So, the classifier is trained on two minibatches of data.

### 2.2 Generator Cost $J^{(G)}$
There are many ways to define a cost function for the generator, based on the specific task at hand. 
The simplest version of the generator cost is the opposite of the discriminator's:
\begin{equation*}
J^{(G)} = - J^{(D)}.
\end{equation*}
This is also knows as a *zero-sum* game, in which the sum of all players' costs is always zero.
 
 ## 3. Game Design
 ### 3.1. Minimax
Using the *zero-sum* design, the optimization problem becomes a minimax problem because the solution involves minimization in an outer loop and maximization in an inner loop:
\begin{equation*}
\mathbf{\theta}^{(G)*} = \text{argmin}_{\mathbf{\theta}^{(G)}}\text{max}_{\mathbf{\theta}^{(D)}} - J^{(D)}(\mathbf{\theta}^{(D)}, \mathbf{\theta}^{(G)})
\end{equation*}
- This version of the game has nice propertiesfor theoretical analysis but does not perform well in practice.
  - Learning resembles minimizing the Jensen-Shannon divergence between the data and the model distribution.
  - Game converges to its equilibrium if both players' policies can be updated directly in function space.
  - In practice, players are deep neural nets and updates are made in parameter space. So, the results do not apply.

### 3.2 Heuristic Non-saturating Game
In the minimiax game, discriminator minimizes a cross-entropy and the generator maximizes the same-cross entropy. This setting is problematic because when the discriminator acquires low loss (near zero), the generator's gradients will vanish (simply flipping the sign of a near zero value won't give us a good error).

Instead of flipping the sign on the discriminator's cost to obtain a cost for the generator, we flip the target used to construct the cross-entropy cost. Then, the generator cost becomes:
\begin{equation*}
J^{(G)} = -\frac{1}{2}\mathbb{E}_{\mathbf{z}}\log D(G(z)).
\end{equation*}
**What is the difference?**
- In the minimax game, the generator minimizes the log-probability of the discriminator being correct.
- In this game, the generator maximizes the log probability of the discriminator being mistaken.
- This version of the game is a heuristic modification to the original, therefore it is no longer zero-sum. **However, it ensures that each player has a strong gradient when that player is losing the game.**

### 3.3. Maximum Likelihood Game
It is also possible to maximize likelihood, or minimize the KL-Divergence between the data and the model. There are variety of methods to do that, however we omit this for now because heuristic non-saturating game has shown to perform better. For details of the maximum likelihood game and why it fails, check page 26, figure 16 of the NIPS tutorial on GANs by Ian Goodfellow.

## 4. Tips and Tricks
It is hard to train GANs. We explore some tips and tricks that can possibly help us. For more, check this NIPS workshop on how to train GANS: 
https://github.com/soumith/ganhacks.
### 4.1. Train with Labels
1. Using labels always results in a dramatic improvement (https://www.kdd.org/kdd2019/accepted-papers/view/learning-class-conditional-gans-with-active-sampling).
2. Training **only** the discriminator on labeled data improves the quality (https://arxiv.org/pdf/1606.03498.pdf).

### 4.2. One-sided Label Smoothing
DNNs are prone to producing highly confident outputs that identify the correct class but with too extreme of a probability. One-sided label smoothing punishes extremely high confidence predictions if the sample was not fake. This acts as a **regularizer** because it does not encourage the model to choose an incorrect class on the training set, but only to reduce the confidence in the correct class (standard regularizers punish confidence that may encourage incorrect classifications).

### 4.3. Virtual Batch Normalization
Batch normalization is used to improve optimization of the model, by reparameterizing the model so that the mean and variance of each feature are determined by a single mean parameter and a single variance parameter associated with that feature. This simplifies the interactions between weights, because otherwise a complex interaction between all of the weights of all of the layers would determine the mean and variances.

### 4.4 Balancing $G$ and $D$
- Some researchers think it is important to balance the training of the generator and the discriminator powers. Sometimes when discriminator starts to acqiure very low loss, the gradient for the generator can vanish because the discriminator is too accurrate. **The right way to solve this problem is not to limit the power of the discriminator, but to use a parameterization of the game where the gradient does not vanish.**

- To balance the opposite case, where the gradient for the generator becomes very large because the discriminator is too confident, **use label smoothing.**

- Another way to balance the generator and discriminator, we can choose different model sizes. In practice, the discriminator is usually deeper and sometimes has more filters per layer than the generator. **Reasons can be:**
  - It is important for discriminator to be correct.
  - Artifact of the mode collapse problem: generators tend not to use their full capcity with current training methods. So, there is no point to increase their capacity.

## 5. Research Frontiers
### 5.1. Non-convergence
- Optimization algorithms usually make downhill progress in the loss surface. On the other hand because GANs play a minimax type of game, each update can direct the optimization towards or away from the equillibrium.
- There is no theoretical proof that GANs should converge, using the gradient based update rules. In practice, GANs often seem to oscillate: they progress from generating one kind of sample from another kind without reaching an equilibrium.

#### 5.1.1. Mode Collapse
- The most common form of harmful non-convergence. This  is a problem that occurs when the generator learns to map several different input $z$ values to the same output point.

- In practice, complete mode collapse is rare, but partial mode collapse is commmon. E.g., generator makes multiple images that contain the same color or texture themes, multiple images containing different views of the same dog.

- Mode collapse may arise because simultaneous gradient descent does not clearly privilege the minmax over maxmin. It ofthen behaves like it is solving the maxmin problem.

- Potential solutions attemps that work fairly well for mode collapse are:
minibatch features (Salimans et al., 2016), unrolled GANs (Metz et al., 2016).

#### 5.1.2 Other Stuff
Some other important things to mention are:
1. There is no clear evaluation metric for generated samples for GANs. Models that obtain good likelihood can generate bad samples, models that generate good samples can have poor likelihood. GANs are especially harder because it can be difficult to estimate the likelihood of GANs.

2. Discrete outputs. Generators must be differentiable. Therefore, generator can not produce discrete data such as one-hot word or character representations. Some possible ways to address this are:
  - Using the REINFORCE algorithm,
  - Using the concrete distribution or Gumbel-softmax,
  - Training the generate to sample continuous values that can be decoded to discrete ones (sampling word embeddings directly).

3. State-of-the-art **semi-supervised** learner is a GAN (Salimans et al., 2016).



